In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from PIL import Image
from IPython.core.display import HTML
from functools import partial
import cv2

from visprog.engine.utils import ProgramGenerator, ProgramInterpreter
from visprog.prompts.clevrer import create_prompt
import base64
import numpy as np
from tqdm import tqdm

[2024-01-17 17:20:44,059] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
module_list = ["LOC", "TRACK", "GRAY", "RED", "BLUE", "GREEN", "CUBE", "SPHERE", "RUBBER", "LEFT", "TOP", "LEFTOF", "FRONTOF", "EVENT", "BEFORE", "EVAL", "RESULT"]
use_precomputed = False
interpreter = ProgramInterpreter(dataset='clevrer', use_precomputed=use_precomputed, module_list=module_list)


Registering LOC step
Registering TRACK step
Registering LEFT step
Registering RIGHT step
Registering TOP step
Registering BOTTOM step
Registering GRAY step
Registering RED step
Registering BLUE step
Registering GREEN step
Registering BROWN step
Registering PURPLE step
Registering CYAN step
Registering YELLOW step
Registering CUBE step
Registering SPHERE step
Registering CYLINDER step
Registering RUBBER step
Registering METAL step
Registering LEFTOF step
Registering RIGHTOF step
Registering FRONTOF step
Registering BEHIND step
Registering EVENT step
Registering BEFORE step
Registering EVAL step
Registering RESULT step


In [4]:
prog = """OBJ0=LOC(video=VIDEO,object='object')
OBJT=TRACK(object=OBJ0)
PRED0=RED(object=OBJT,var='o0')
PRED1=RUBBER(object=OBJT,var='o0')
PRED2=TOP(object=OBJT,var='o1')
PRED3=FRONTOF(object1=OBJT,var1='o0',object2=OBJT,var2='o1')
EVENT0=EVENT(predicates=[PRED0,PRED1,PRED2,PRED3],min_duration=1)
PRED4=FRONTOF(object1=OBJT,var1='o1',object2=OBJT,var2='o0')
EVENT1=EVENT(predicates=[PRED4],min_duration=1)
EVENT2=BEFORE(event1=EVENT0, event2=EVENT1)
PRED5=TOP(object=OBJT,var='o2')
PRED6=LEFT(object=OBJT,var='o2')
EVENT3=EVENT(predicates=[PRED5,PRED6],min_duration=25)
EVENT4=BEFORE(event1=EVENT2, event2=EVENT3)
ANSWER0=EVAL(expr="'yes' if len({EVENT4}) else 'no'")
FINAL_RESULT=RESULT(var=ANSWER0)"""

In [5]:
output = []
for vid in tqdm([11]):
    if not use_precomputed:
        cap = cv2.VideoCapture(
            os.path.join(
                '/gscratch/balazinska/enhaoz/VOCAL-UDF/data',
                'clevrer', 
                f'video_{str(vid//1000*1000).zfill(5)}-{str((vid//1000+1)*1000).zfill(5)}', 
                f"video_{str(vid).zfill(5)}.mp4"
            )
        )
        video = []
        fid = 0
        while True:
            ret, frame = cap.read()  # Read the next frame from the video
            if not ret:
                break  # Break the loop if there are no more frames

            # Convert the frame from BGR (OpenCV format) to RGB (PIL format)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to a PIL Image and append to the list
            image = Image.fromarray(frame)
            image.thumbnail((640,640),Image.Resampling.LANCZOS)
            video.append(image)
            fid += 1
    init_state = dict(
        VIDEO=video,
        vid=vid
    )
    result, prog_state = interpreter.execute(prog,init_state,inspect=False)
    if result == 'yes':
        output.append(vid)
print(output)

  0%|                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

LOC
TRACK
RED
RUBBER
TOP
FRONTOF
EVENT
FRONTOF
EVENT
BEFORE
TOP


/mmfs1/gscratch/balazinska/enhaoz/VOCAL-UDF/ByteTrack/yolox/tracker/byte_tracker.py:129: RuntimeWarning: divide by zero encountered in double_scalars
  ret[2] /= ret[3]
/mmfs1/gscratch/balazinska/enhaoz/VOCAL-UDF/ByteTrack/yolox/tracker/byte_tracker.py:107: RuntimeWarning: invalid value encountered in double_scalars
  ret[2] *= ret[3]
/mmfs1/gscratch/balazinska/enhaoz/VOCAL-UDF/ByteTrack/yolox/tracker/byte_tracker.py:107: RuntimeWarning: invalid value encountered in double_scalars
  ret[2] *= ret[3]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.50s/it]

LEFT
EVENT
BEFORE
EVAL
RESULT
[]
